<a href="https://colab.research.google.com/github/KhazaeiM/Anomaly_detection/blob/main/04__RandomWalks_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
import pandas as pd
import numpy as np
import itertools
import csv
from itertools import chain
import random
import functools
import operator
from collections import Counter

In [ ]:
data_path = "/content/drive/MyDrive/data/Thesise Data/AMiner_Academic/"
dimen = 128
window = 5
walk_length = 100
a_top = 10
p_top = 10
v_top = 3

In [ ]:
papers = pd.read_csv(data_path + 'AMiner-Papers-inj-200-train-csv.csv', index_col=False)
authors = pd.read_csv(data_path + 'AMiner-Authors-inj-200-train-csv.csv', index_col=False)
venues = pd.read_csv(data_path + 'AMiner-Venue-A&P-inj-200-train-csv.csv', index_col=False)
contribution  = pd.read_csv(data_path + 'AMiner-A&A-inj-200-train-csv.csv', index_col=False)

In [ ]:
len(papers['venue_name'])

262180

In [ ]:
papers['venue_name'].value_counts()

['-']       244211
['v48']       1886
['v202']      1077
['v203']       907
['v10']        848
             ...  
['v199']         3
['v195']         3
['v58']          3
['v19']          1
['v21']          1
Name: venue_name, Length: 161, dtype: int64

In [ ]:
# CONVERT str TO list
def str2list(df, col):
  df[col] = df[col].transform(lambda x: x.replace('[', "").replace(']', ""))
  df[col] = df[col].apply(lambda x: [j for i in csv.reader(x, delimiter = ',', quotechar="'") for j in i if j.strip()])

In [ ]:
# SPLIT EACH CLIQUE TO PAPER AND AUTHOR NODES
def separated(lis):
  a_list = []
  p_list = []
  v_list = []

  for j in range(len(lis)):
    if lis[j][0] == 'a':
      a_list.append(lis[j])
    if lis[j][0] == 'p':
      p_list.append(lis[j])
    if lis[j][0] == 'v':
      v_list.append(lis[j])
  return (a_list, p_list, v_list)

In [ ]:
v_cols = ['index_a', 'index_p']
for col in v_cols:
  str2list(venues, col)
a_cols = ['index_p', 'venue_name', 'co_authors']
for col in a_cols:
  str2list(authors, col)
p_cols = ['index_a', 'references', 'venue_name']
for col in p_cols:
  str2list(papers, col)

In [ ]:
papers['index_a'] = papers['index_a'].transform(lambda x: '' if x == ['-'] else x)
papers['references'] = papers['references'].transform(lambda x: '' if x == ['-'] else x)
papers['venue_name'] = papers['venue_name'].transform(lambda x: '' if x == ['-'] else x)
papers.head(3)

,index_p,index_a,year,references,venue_name,abnormal
0,p1000022,[a1641825],2007,,,0
1,p1000023,[a1642199],2007,,,0
2,p1000028,"[a1558799, a1654966, a830738]",2007,,,0


In [ ]:
authors['index_p'] = authors['index_p'].transform(lambda x: '' if x == ['-'] else x)
authors['co_authors'] = authors['co_authors'].transform(lambda x: '' if x == ['-'] else x)
authors['venue_name'] = authors['venue_name'].transform(lambda x: '' if x == ['-'] else x)
authors.head(3)

,index_a,index_p,interest,venue_name,abnormal,co_authors
0,a100,[p1853556],"['networked', 'technology', 'work', 'practice'...",,0,
1,a1000000,[p1608103],"['asynchronous', 'task', 'OpenMP', 'implementa...",,0,"[a1145211, a937524]"
2,a1000005,[p1613472],"['automatic', 'segmentation', 'automatic', 've...",,0,


In [ ]:
venues = venues[1:].reset_index(drop=True)
venues.head(3)

,venue_name,index_p,index_a
0,v1,"[p1304711, p954849, p2034776, p1002497, p16793...","[a1617363, a1085914, a1453204, a1289832, a1007..."
1,v10,"[p2012819, p1526732, p1224795, p1911661, p1960...","[a1006833, a114793, a1641427, a488688, a105204..."
2,v100,"[p1476514, p1476445, p1476500, p1476455, p1476...","[a953745, a1282951, a1284117, a1284426, a12411..."


In [ ]:
contribution['auth'] = contribution['auth'].apply(lambda x: x.replace("(", "").replace(")", "").replace("'", "").replace(" ", ""))
contribute = pd.DataFrame(contribution.auth.str.split(',',1).tolist(), columns = ['auth_1','auth_2'])
contribute['counts'] =  contribution['counts']
contribute.head()

,auth_1,auth_2,counts
0,a111806,a977442,246
1,a111806,a966551,246
2,a966551,a977442,246
3,a215655,a529794,72
4,a1162933,a839667,57


In [ ]:
def random_choice(series, idx_current):
  if (len(series.iloc[idx_current]) != 0):
    return random.choice(series.iloc[idx_current])

In [ ]:
# SPLIT EACH CLIQUE TO PAPER AND AUTHOR NODES
def separated(lis):
  a_list = []
  p_list = []
  v_list = []

  for j in range(len(lis)):
    if lis[j][0] == 'a':
      a_list.append(lis[j])
    if lis[j][0] == 'p':
      p_list.append(lis[j])
    if lis[j][0] == 'v':
      v_list.append(lis[j])
  return (a_list, p_list, v_list)

In [ ]:
def sampled(a_list, a_sample, p_list, p_sample, v_list, v_sample, start_node):

  def top_k(common, common_1, common_2, k):
    top_node = []
    if len(common) == 0:
      lis = common_1 + common_2
      common = np.random.choice(lis, k)

    for item in common:
      top_node.append(item)
    while len(top_node) != k:
      ch = random.choice(common)
      top_node.append(ch)
    return top_node

  common_a = [x[0] for x in Counter(a_list).most_common(a_sample)]
  common_p = [x[0] for x in Counter(p_list).most_common(p_sample)]
  common_v = [x[0] for x in Counter(v_list).most_common(v_sample)]

  sample = [start_node]
  top_a = top_k(common_a, common_p, common_v, a_sample)
  sample.extend(top_a)
  top_p = top_k(common_p, common_a, common_v, p_sample)
  sample.extend(top_p)
  top_v = top_k(common_v, common_p, common_a, v_sample)
  sample.extend(top_v)
  return(sample)


In [ ]:
len(authors['index_a'])

227834

In [ ]:
phase = authors[16000:20000]

neigh_auth = ['author', 'paper', 'venue']
neigh_paper = ['author', 'paper', 'venue']
neigh_venue = ['author', 'paper']
types = [authors, papers, venues]
col_index = ['index_a', 'index_p', 'venue_name']

with open(data_path+'AMiner_Walks_a_20000.txt', 'w') as textfile:
  j = 0
  for idx, rows in phase.iterrows():
    node_walk = []
    start = rows['index_a']
    current = start
    node_walk.append(current)
    while len(node_walk) < walk_length:
      prob = random.random()
      if prob > 0.5:
        if current[0] == 'a':
          idx_current = types[0].index[authors['index_a'] == current][0]
          choose = random.choice(neigh_auth)
          if choose == 'author':
            current = random_choice(authors['co_authors'], idx_current)
          elif choose == 'paper':
            current = random_choice(authors['index_p'], idx_current)
          elif choose == 'venue':
            current = random_choice(authors['venue_name'], idx_current)

        elif current[0] == 'p':
          idx_current = papers.index[papers['index_p'] == current][0]
          choose = random.choice(neigh_paper)
          if choose == 'author':
            current = random_choice(papers['index_a'], idx_current)
          elif choose == 'paper':
            current = random_choice(papers['references'], idx_current)
          elif choose == 'venue':
            current = random_choice(papers['venue_name'], idx_current)

        elif current[0] == 'v':
          idx_current = venues.index[venues['venue_name'] == current][0]
          choose = random.choice(neigh_venue)
          if choose == 'author':
            current = random_choice(venues['index_a'], idx_current)
          elif choose == 'paper':
            current = random_choice(venues['index_p'], idx_current)

      else:
        current = start
      if current == None:
        current = node_walk[-1]
      node_walk.append(current)
        # print(current)    
    temp = []
    for i in range(len(node_walk)-1):
      temp.append(node_walk[i])
      if (node_walk[i][0] == 'a') and (node_walk[i+1][0] == 'a'):
        filter = (contribute['auth_1'] == node_walk[i]) & (contribute['auth_2'] == node_walk[i+1])
        idx = contribute[filter].index.values
        cont = contribute['counts'].iloc[idx].values
        if len(cont) != 0:
          repeated = cont[0]
          temp.extend([node_walk[i+1]]*(repeated - 1))

    (a_list, p_list, v_list) = separated(temp)
    samples = sampled(a_list, a_top, p_list, p_top, v_list, v_top, node_walk[0])
    j += 1
    # print(samples)
    textfile.write(str(samples) + "\n")
    if j%100 == 0:
      print(j)
textfile.close()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [ ]:
# textfile = open(data_path + "AMiner_Walks.txt", "w")

# neigh_auth = ['author', 'paper', 'venue']
# neigh_paper = ['author', 'paper', 'venue']
# neigh_venue = ['author', 'paper']
# types = [authors, papers, venues]
# col_index = ['index_a', 'index_p', 'venue_name']
# all_walks = []

# for i in range(3):
#   type_walk = []
#   for idx, rows in types[i].iterrows():
#     node_walk = []
#     start = rows[col_index[i]]
#     current = start
#     node_walk.append(current)
#     while len(node_walk) < walk_length:
#       prob = random.random()
#       if prob > 0.5:
#         if current[0] == 'a':
#           idx_current = types[0].index[authors['index_a'] == current][0]
#           choose = random.choice(neigh_auth)
#           if choose == 'author':
#             current = random_choice(authors['co_authors'], idx_current)
#           elif choose == 'paper':
#             current = random_choice(authors['index_p'], idx_current)
#           elif choose == 'venue':
#             current = random_choice(authors['venue_name'], idx_current)

#         elif current[0] == 'p':
#           idx_current = papers.index[papers['index_p'] == current][0]
#           choose = random.choice(neigh_paper)
#           if choose == 'author':
#             current = random_choice(papers['index_a'], idx_current)
#           elif choose == 'paper':
#             current = random_choice(papers['references'], idx_current)
#           elif choose == 'venue':
#             current = random_choice(papers['venue_name'], idx_current)

#         elif current[0] == 'v':
#           idx_current = venues.index[venues['venue_name'] == current][0]
#           choose = random.choice(neigh_venue)
#           if choose == 'author':
#             current = random_choice(venues['index_a'], idx_current)
#           elif choose == 'paper':
#             current = random_choice(venues['index_p'], idx_current)

#       else:
#         current = start
#       if current == None:
#         current = node_walk[-1]
#       node_walk.append(current)
#       # print(current)    
#     type_walk.append(node_walk)
#     textfile.write(str(node_walk) + "\n")
#     print(len(type_walk))
#     # print(type_walk)
#   print('***********************')
#   all_walks.append(type_walk)
# textfile.close()

In [ ]:
  # all_walks[2][526]

In [ ]:
# textfile = open(data_path + "AMiner_Walks.txt", "w")
# for element in all_walks:
#   textfile.write(element + "\n")
# textfile.close()

In [ ]:
# venues[venues['venue_name'] == 'v12']

In [ ]:
# venues.head(100)

In [ ]:
# authors[authors['index_a'] == 'a1000027']